# START A H2O CLUSTER


In [2]:
%matplotlib inline
import h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM 18.3 (build 10.0.2+13, mixed mode)
  Starting server from C:\Users\bhavy\Anaconda3\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\bhavy\AppData\Local\Temp\tmpdk132jz6
  JVM stdout: C:\Users\bhavy\AppData\Local\Temp\tmpdk132jz6\h2o_bhavy_started_from_python.out
  JVM stderr: C:\Users\bhavy\AppData\Local\Temp\tmpdk132jz6\h2o_bhavy_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,03 secs
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.1.6
H2O cluster version age:,8 days
H2O cluster name:,H2O_from_python_bhavy_27serx
H2O cluster total nodes:,1
H2O cluster free memory:,3.961 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


# IMPORT THE DATASET

path ---> The complete URL or normalized file path of the file to be imported. Each row of data appears as one line of the file.

destination_frame ---> (Optional) The unique hex key assigned to the imported file. If none is given, a key will automatically be generated based on the URL path. parse

In [5]:
destination_frame_name = 'lending_club_data.hex'
lc_loan_data = h2o.import_file(path=r'C:\Users\bhavy\Desktop\INFO_DS\Project3\Cleaned_loan.csv', destination_frame=destination_frame_name)

Parse progress: |█████████████████████████████████████████████████████████| 100%


# EXPLORATORY DATA ANALYSIS IN H2O

# FEATURE ENGINEERING IN H2O

In order to predict the loan defaults we create two variables y which hold the response column bad_loan and x_orig hold the list of predictors. One of the predictors is the int_rate as well since this value is set based on the risk assesment of the loan lended. 

Some of the feature engineering tasks performed are:
1.    Converting Term to a 0/1 Indicator
2.    Combining Categories in Home Ownership

In [6]:
y = "int_rate"
x_orig = lc_loan_data.col_names
x_orig.remove(y)
#x_orig.remove("int_rate")

x_trans = x_orig.copy()

In [7]:
def mean_target(data, x, y = "int_rate"):
    grouped_data = data[[x, y]].group_by([x])
    stats = grouped_data.count(na = "ignore").mean(na = "ignore")
    return stats.get_frame().as_data_frame()

In [8]:
import numpy as np
import pandas as pd

CONVERTING TERM TO A 0/1 INDICATOR

Given that term of the loans are either 3 or 5 years.
We will create a simplifed term_n_month binary indicator that is 1 when the terms of the loan is for 5 years and 0 for loans with a term of 3 years.

In [9]:
lc_loan_data["term"].table()

term,Count
36,621125
60,266254


In [10]:
x_trans.remove("term")
x_trans.append("term_n_months")

lc_loan_data["term_n_months"] = lc_loan_data["term"] == "60 months"

In [11]:
lc_loan_data["term_n_months"].table()

term_n_months,Count
0,887379


COMBINING CATEGORIES IN HOME OWNERSHIP

There are 6 recorded categories within home ownership, only three has over 200 observations: OWN, MORTGAGE, and RENT. The remaining three are so infrequent we will combine them {ANY, NONE, OTHER} with RENT to form an enlarged OTHER category. This new home_ownership_3cat variable will have values in {MORTGAGE, OTHER, OWN}.

In [12]:
mean_target(lc_loan_data, "home_ownership")

,home_ownership,nrow,mean_int_rate
0,1,3,14.233333
1,2,443557,12.978171
2,3,50,14.474200
3,4,182,13.462143
4,5,87470,13.266728
5,6,356117,13.576052


In [13]:
lvls = ["OTHER", "MORTGAGE", "OTHER", "OTHER", "OWN", "OTHER"]
lc_loan_data["home_ownership_3cat"] = lc_loan_data["home_ownership"].set_levels(lvls).ascharacter().asfactor()

In [62]:
lc_loan_data[["home_ownership", "home_ownership_3cat"]].table()

H2OResponseError: Server error java.lang.IllegalArgumentException:
  Error: Vector must be a factor column. Got: Numeric
  Request: POST /99/Rapids
    data: {'ast': "(tmp= py_19_sid_9eec (table (cols_py (append Key_Frame__upload_bfc250f46ae6129dff384fe6c3454256.hex (== (cols_py Key_Frame__upload_bfc250f46ae6129dff384fe6c3454256.hex 'term') '60 months') 'term_n_months' (as.factor (as.character (setDomain (cols_py (append Key_Frame__upload_bfc250f46ae6129dff384fe6c3454256.hex (== (cols_py Key_Frame__upload_bfc250f46ae6129dff384fe6c3454256.hex 'term') '60 months') 'term_n_months') 'home_ownership') False ['OTHER' 'MORTGAGE' 'OTHER' 'OTHER' 'OWN' 'OTHER']))) 'home_ownership_3cat') ['home_ownership' 'home_ownership_3cat']) True))", 'session_id': '_sid_9eec'}


In [63]:
mean_target(lc_loan_data, "home_ownership_3cat")

H2OResponseError: Server error java.lang.IllegalArgumentException:
  Error: Vector must be a factor column. Got: Numeric
  Request: POST /99/Rapids
    data: {'ast': "(is.character (cols_py (cols_py (append Key_Frame__upload_bfc250f46ae6129dff384fe6c3454256.hex (== (cols_py Key_Frame__upload_bfc250f46ae6129dff384fe6c3454256.hex 'term') '60 months') 'term_n_months' (as.factor (as.character (setDomain (cols_py (append Key_Frame__upload_bfc250f46ae6129dff384fe6c3454256.hex (== (cols_py Key_Frame__upload_bfc250f46ae6129dff384fe6c3454256.hex 'term') '60 months') 'term_n_months') 'home_ownership') False ['OTHER' 'MORTGAGE' 'OTHER' 'OTHER' 'OWN' 'OTHER']))) 'home_ownership_3cat') ['home_ownership_3cat' 'int_rate']) 1))", 'session_id': '_sid_9eec'}


In [55]:
x_trans.remove("home_ownership")
x_trans.append("home_ownership_3cat")

# Encode response variable

Since we want to train a binary classification model, we must ensure that the response is coded as a factor. If the response is 0/1, H2O will assume it's numeric, which means that H2O will train a regression model instead.

In [82]:
lc_loan_data['int_rate'] = lc_loan_data['int_rate'].ascharacter()

lc_loan_data['int_rate'] = lc_loan_data['int_rate'].asfactor()  #encode the binary repsonse as a factor
lc_loan_data['int_rate'].levels()  #optional: after encoding, this shows the two factor levels, '0' and '1'

[['10',
  '10.01',
  '10.08',
  '10.14',
  '10.15',
  '10.16',
  '10.2',
  '10.25',
  '10.28',
  '10.33',
  '10.36',
  '10.37',
  '10.38',
  '10.39',
  '10.46',
  '10.49',
  '10.51',
  '10.59',
  '10.62',
  '10.64',
  '10.65',
  '10.71',
  '10.74',
  '10.75',
  '10.78',
  '10.83',
  '10.91',
  '10.95',
  '10.96',
  '10.99',
  '11.03',
  '11.09',
  '11.11',
  '11.12',
  '11.14',
  '11.22',
  '11.26',
  '11.28',
  '11.34',
  '11.36',
  '11.41',
  '11.44',
  '11.46',
  '11.48',
  '11.49',
  '11.53',
  '11.54',
  '11.55',
  '11.58',
  '11.59',
  '11.63',
  '11.66',
  '11.67',
  '11.71',
  '11.72',
  '11.78',
  '11.83',
  '11.86',
  '11.89',
  '11.91',
  '11.97',
  '11.99',
  '12.04',
  '12.05',
  '12.09',
  '12.12',
  '12.17',
  '12.18',
  '12.21',
  '12.22',
  '12.23',
  '12.29',
  '12.35',
  '12.36',
  '12.39',
  '12.41',
  '12.42',
  '12.49',
  '12.53',
  '12.54',
  '12.59',
  '12.61',
  '12.62',
  '12.67',
  '12.68',
  '12.69',
  '12.72',
  '12.73',
  '12.8',
  '12.84',
  '12.85',
  '1

# Partition data

Next, we partition the data into training, validation and test sets.

In [83]:
# Partition data into 70%, 15%, 15% chunks
# Setting a seed will guarantee reproducibility

splits = lc_loan_data.split_frame(ratios=[0.7, 0.15], seed=5)  

train = splits[0]
valid = splits[1]
test = splits[2]

In [84]:
train.head()

C1,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,addr_state,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_int,recoveries,collection_recovery_fee,last_pymnt_amnt,collections_12_mths_ex_med,mths_since_last_major_derog,annual_inc_joint,dti_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,term_n_months
1,1.07743e+06,2500,2500,2500,60,15.27,59.83,3,10,6,30000,2,22,1,1,11,1,0,5,0,0,3,0,1687,9.4,4,0,0,1008.71,1008.71,435.17,117.08,1.11,119.66,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1.07686e+06,10000,10000,10000,36,13.49,339.31,3,10,6,49200,2,22,6,10,5,20,0,1,35,0,10,0,5598,21,37,0,0,12226.3,12226.3,2209.33,0,0,357.48,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.07536e+06,3000,3000,3000,60,12.69,67.79,2,1,6,80000,2,22,2,10,38,17.94,0,0,38,0,15,0,27783,53.9,38,766.9,766.9,3242.17,3242.17,1009.07,0,0,67.79,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,1.07205e+06,3000,3000,3000,36,18.64,109.43,5,9,6,48000,2,22,6,1,5,5.35,0,2,0,0,4,0,8221,87.5,4,0,0,3938.14,3938.14,938.14,0,0,111.34,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,1.07157e+06,5375,5375,5350,60,12.69,121.45,2,10,6,15000,3,22,1,10,44,18.08,0,0,0,0,2,0,9279,36.5,3,0,0,1476.19,1469.34,533.42,269.29,2.52,121.45,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10,1.07008e+06,6500,6500,6500,60,14.65,153.45,3,5,5,72000,1,22,6,3,4,16.12,0,2,0,0,14,0,4032,20.6,23,0,0,7677.52,7677.52,1177.52,0,0,1655.54,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
11,1.06991e+06,12000,12000,12000,36,12.69,402.54,2,10,5,75000,2,22,6,3,5,10.78,0,0,0,0,12,0,23336,67.1,34,0,0,13943.1,13943.1,1943.08,0,0,6315.3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
12,1.06469e+06,9000,9000,9000,36,13.49,305.38,3,10,6,30000,2,22,1,3,46,10.08,0,1,0,0,4,0,10452,91.7,9,0,0,2270.7,2270.7,570.26,444.3,4.16,305.38,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
13,1.06987e+06,3000,3000,3000,36,9.91,96.68,2,3,6,15000,2,22,6,2,15,12.56,0,2,0,0,11,0,7323,43.1,11,0,0,3478.98,3478.98,478.98,0,0,102.43,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
14,1.06906e+06,10000,10000,10000,36,10.65,325.74,2,3,6,100000,2,22,1,10,5,7.06,0,2,0,0,14,0,11997,55.5,29,0,0,7471.99,7471.99,1393.42,645.1,6.3145,325.74,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Notice that split_frame() uses approximate splitting not exact splitting (for efficiency), so these are not exactly 70%, 15% and 15% of the total rows.

In [85]:
print (train.nrow)
print (valid.nrow)
print (test.nrow)

621334
133178
132867


# Identify response and predictor variables

In H2O, we use y to designate the response variable and x to designate the list of predictor columns

In [86]:
y = 'int_rate'
x = list(lc_loan_data.columns)

In [87]:
x.remove(y)  #remove the response

# H2O MACHINE LEARNING


Now that we have prepared the data, we can train some models. We will start by training a single model from each of the H2O supervised algos:

  1. Generalized Linear Model (GLM)
  2. Random Forest (RF)
  3. Deep Learning (DL)

1. GENERALIZED LINEAR MODEL

Let's start with a basic binomial Generalized Linear Model (GLM). By default, H2O's GLM uses a regularized, elastic net model.

In [88]:
# Import H2O GLM:
from h2o.estimators.glm import H2OGeneralizedLinearEstimator

Create an object of class, "H2OGeneralizedLinearEstimator". This does not  do any training, it just sets the model up for training by specifying model parameters.

In [89]:
# Initialize the GLM estimator:

glm_fit = H2OGeneralizedLinearEstimator(family = 'ordinal' ,model_id='glm_fit1')

Now that glm_fit object is initialized, we can train the model:

In [90]:
glm_fit.train(x=x, y=y, training_frame=train)

glm Model Build progress: |███████████████████████████████████████████████| 100%


In [91]:
y_pred=glm_fit.predict(test)

glm prediction progress: |████████████████████████████████████████████████| 100%


In [95]:
from sklearn.metrics import accuracy_score
sklearn.metrics.accuracy_score(y, y_pred, normalize=True, sample_weight=None)

NameError: name 'sklearn' is not defined

2. RANDOM FOREST

H2O's Random Forest (RF) is implemented as a distributed version of the standard Random Forest algorithm and variable importance measures.

In [96]:
# Import H2O RF:
from h2o.estimators.random_forest import H2ORandomForestEstimator

First we  train a basic Random Forest model with default parameters.
Random Forest will infer the response distribution from the response encoding.
A seed is required for reproducibility.

In [97]:
# Initialize the RF estimator:

rf_fit = H2ORandomForestEstimator(model_id='rf_fit', seed=1)

In [98]:
train.head

C1,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,addr_state,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_int,recoveries,collection_recovery_fee,last_pymnt_amnt,collections_12_mths_ex_med,mths_since_last_major_derog,annual_inc_joint,dti_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,term_n_months
1,1.07743e+06,2500,2500,2500,60,15.27,59.83,3,10,6,30000,2,22,1,1,11,1,0,5,0,0,3,0,1687,9.4,4,0,0,1008.71,1008.71,435.17,117.08,1.11,119.66,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1.07686e+06,10000,10000,10000,36,13.49,339.31,3,10,6,49200,2,22,6,10,5,20,0,1,35,0,10,0,5598,21,37,0,0,12226.3,12226.3,2209.33,0,0,357.48,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.07536e+06,3000,3000,3000,60,12.69,67.79,2,1,6,80000,2,22,2,10,38,17.94,0,0,38,0,15,0,27783,53.9,38,766.9,766.9,3242.17,3242.17,1009.07,0,0,67.79,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,1.07205e+06,3000,3000,3000,36,18.64,109.43,5,9,6,48000,2,22,6,1,5,5.35,0,2,0,0,4,0,8221,87.5,4,0,0,3938.14,3938.14,938.14,0,0,111.34,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,1.07157e+06,5375,5375,5350,60,12.69,121.45,2,10,6,15000,3,22,1,10,44,18.08,0,0,0,0,2,0,9279,36.5,3,0,0,1476.19,1469.34,533.42,269.29,2.52,121.45,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10,1.07008e+06,6500,6500,6500,60,14.65,153.45,3,5,5,72000,1,22,6,3,4,16.12,0,2,0,0,14,0,4032,20.6,23,0,0,7677.52,7677.52,1177.52,0,0,1655.54,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
11,1.06991e+06,12000,12000,12000,36,12.69,402.54,2,10,5,75000,2,22,6,3,5,10.78,0,0,0,0,12,0,23336,67.1,34,0,0,13943.1,13943.1,1943.08,0,0,6315.3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
12,1.06469e+06,9000,9000,9000,36,13.49,305.38,3,10,6,30000,2,22,1,3,46,10.08,0,1,0,0,4,0,10452,91.7,9,0,0,2270.7,2270.7,570.26,444.3,4.16,305.38,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
13,1.06987e+06,3000,3000,3000,36,9.91,96.68,2,3,6,15000,2,22,6,2,15,12.56,0,2,0,0,11,0,7323,43.1,11,0,0,3478.98,3478.98,478.98,0,0,102.43,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
14,1.06906e+06,10000,10000,10000,36,10.65,325.74,2,3,6,100000,2,22,1,10,5,7.06,0,2,0,0,14,0,11997,55.5,29,0,0,7471.99,7471.99,1393.42,645.1,6.3145,325.74,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


<bound method H2OFrame.head of >

Now that rf_fit object is initialized, we can train the model:

In [100]:
rf_fit.train(x=x, y=y, training_frame=train)

TypeError: string indices must be integers

3. DEEP LEARNING (DL)

H2O's Deep Learning algorithm is a multilayer feed-forward artificial neural network. It can also be used to train an autoencoder, however, in the example below we will train a standard supervised prediction model.

In [82]:
# Import H2O DL:
from h2o.estimators.deeplearning import H2ODeepLearningEstimator

The DL infers the response distribution from the response encoding if not specified explicitly through the distribution argument.
H2O's DL will not be reproducbible if run on more than a single core, so in this example, the performance metrics below may vary slightly.

In [83]:
# Initialize and train the DL estimator:

dl_fit = H2ODeepLearningEstimator(model_id='dl_fit', seed=1)
dl_fit.train(x=x, y=y, training_frame=train)

deeplearning Model Build progress: |█████████████████████████ (cancelled)


H2OJobCancelled: Job<$03017f00000132d4ffffffff$_98ac4bfcdc11808d657d821c6e784cf1> was cancelled by the user.

# EVALUATE MODEL PERFORMANCE

Comparing  the performance of the GLM , RF , DL models that were just trained.

In [96]:
perf1 = glm_fit.model_performance(test)
#perf2 = rf_fit.model_performance(test)
#perf3 = dl_fit.model_performance(test)

In [98]:
str(perf1)

# Retreive test set AUC(Area under the curve)
#print (perf1.auc())
#print (perf2.auc())
#print (perf3.auc())


ModelMetricsOrdinalGLM: glm
** Reported on test data. **

MSE: 0.9904310184611638
RMSE: 0.9952040084631713


''

In [45]:
h2o.cluster().shutdown(prompt=False)

H2O session _sid_9e4f closed.
